In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lmfit import minimize, Parameters, fit_report, Model

### Defining the models and origins

In [23]:
def fn(x,a,m):
    xdep = (-27*x*m/2/a+np.sqrt(729*(x*m/a)**2+108/a**3)/2)
    return xdep**(-1./3)/a-xdep**(1./3)/3
def fngrad(x,a,m):
    return m/(1+3*a*fn(x,a,m)**2)
def fnc1(x,a,m1):
        return fn(x-xl[0],a,m1)+yl[0]
def fnc2(x,a,m1,m2,p,q):
        return p*(fn(x-xl[0],a,m1)+yl[0])+q*(fn(x-xl[1],a,m2)+yl[1])
def fnc3(x,a,m1,m2,m3,p,q,r):
        return p*(fn(x-xl[0],a,m1)+yl[0])+q*(fn(x-xl[1],a,m2)+yl[1])+r*(
            fn(x-xl[2],a,m3)+yl[2])
def fnc4(x,a,m1,m2,m3,m4,p,q,r,s):
    return p*(fn(x-xl[0],a,m1)+yl[0])+q*(fn(x-xl[1],a,m2)+yl[1])+r*(
            fn(x-xl[2],a,m3)+yl[2])+s*(fn(x-xl[3],a,m4)+yl[3])
def origins(x,y,ams):
    grads=(y[1:]-y[:-1])/(x[1:]-x[:-1])
    mxpts = np.argpartition(np.abs(grads),-ams-1)[-ams-1:]
    xl = np.empty(ams+1)
    yl = np.empty(ams+1)
    for i in np.arange(len(mxpts)):
        xl[i] = (x[mxpts[i]]+x[mxpts[i]+1])/2
        yl[i] = (y[mxpts[i]]+y[mxpts[i]+1])/2
    return xl,yl,grads[mxpts]

def add_ams(n,grads):
    mi1,mi2,mi3,mi4=[-1,1,1,-1]
    if n==0:model = Model(fnc1);#mi1=grads[0]
    elif n==1:model = Model(fnc2);#mi2,mi1 = grads
    elif n==2:model = Model(fnc3);#mi3,mi2,mi1 = grads
    elif n==3:model = Model(fnc4);#mi4,mi3,mi2,mi1 = grads
    parms = model.make_params(a=dict(value=1, min=1e-3),
                               m1=dict(value=mi1),m2=dict(value=mi2),
                                 m3=dict(value=mi3),m4=dict(value=mi4),
                                 p=dict(value=1),q=dict(value=-1),
                                 r=dict(value=1),s=dict(value=-1),)
    return model,parms

### Loading the data and estimating model parameters

In [24]:
df = pd.read_table('bactgrowth.txt')
idx = df[df['time']==0].index
for k in range(4):
    res = [];xc=[];yc=[]
    for j in range(72):
        x = df[idx[j]:idx[j]+31]['time'].to_numpy()
        y = df[idx[j]:idx[j]+31]['value'].to_numpy()
        detls = {'strain':df.iloc[idx[j]].strain, 'replicate':df.iloc[idx[j]].replicate,
                 'conc':df.iloc[idx[j]].conc}
        xl,yl,grads = origins(x,y,k)
        xc.append(xl);yc.append(yl)
        model,parms = add_ams(k,grads)
        out = model.fit(y,parms,x=x)
        detls.update(out.values)
        res.append(detls)
    if k==0:
        df_res1 = pd.DataFrame.from_dict(res)
        df_res1['xc1']=[i[0] for i in xc]
        df_res1['yc1']=[i[0] for i in yc]
    elif k==1:
        df_res2 = pd.DataFrame.from_dict(res)
        df_res2['xc1']=[i[0] for i in xc]
        df_res2['xc2']=[i[1] for i in xc]
        df_res2['yc1']=[i[0] for i in yc]
        df_res2['yc2']=[i[1] for i in yc]
    elif k==2:
        df_res3 = pd.DataFrame.from_dict(res)
        df_res3['xc1']=[i[0] for i in xc]
        df_res3['xc2']=[i[1] for i in xc]
        df_res3['xc3']=[i[2] for i in xc]
        df_res3['yc1']=[i[0] for i in yc]
        df_res3['yc2']=[i[1] for i in yc]
        df_res3['yc3']=[i[2] for i in yc]
    elif k==3:
        df_res4 = pd.DataFrame.from_dict(res)
        df_res4['xc1']=[i[0] for i in xc]
        df_res4['xc2']=[i[1] for i in xc]
        df_res4['xc3']=[i[2] for i in xc]
        df_res4['xc4']=[i[3] for i in xc]
        df_res4['yc1']=[i[0] for i in yc]
        df_res4['yc2']=[i[1] for i in yc]
        df_res4['yc3']=[i[2] for i in yc]
        df_res4['yc4']=[i[3] for i in yc]


### Generate csv files

In [ ]:
df_res1=pd.read_csv('NI.csv')
df_res2=pd.read_csv('NII.csv')
df_res3=pd.read_csv('NIII.csv')
df_res4=pd.read_csv('NIV.csv')